# Load the modules

In [1]:
import cupy as cp
import numpy as np
import cudf
import dask_cudf
import cugraph
import cuspatial
import datetime as dttm
from cuml.preprocessing import (LabelEncoder, OneHotEncoder, train_test_split)
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from blazingsql import BlazingContext
from os import listdir

In [2]:
cluster = LocalCUDACluster()
client = Client(cluster)

bc = BlazingContext(dask_client=client)

BlazingContext ready


# Create the tables

In [3]:
transactions_path = '../../data/seattle_parking/parking_MayJun2019.parquet'
transactions_parq = [f'{transactions_path}/{d}' for d in listdir(transactions_path) if d != '.ipynb_checkpoints']

locations_path = '../../data/seattle_parking/parking_locations.parquet'
locations_parq = [f'{locations_path}/{d}' for d in listdir(locations_path) if d != '.ipynb_checkpoints']

In [4]:
bc.create_table('parking_transactions', transactions_parq)
bc.create_table('parking_locations', locations_parq)

# Featurize parking locations

In [5]:
parking_locations = bc.sql('SELECT * FROM parking_locations').compute()

parking_locations['ParkingArea_concat'] = (
    parking_locations['PaidParkingArea']
    .str.replace(' ', '_')
    .str.cat(parking_locations['PaidParkingSubArea'], sep='__')
)

parking_locations['ParkingArea_concat'].head()

0              Belltown__North
1                 Uptown__Edge
2         Pioneer_Square__Core
3               Pike-Pine__UKN
4    University_District__Edge
Name: ParkingArea_concat, dtype: object

In [6]:
le = LabelEncoder()
ohe = OneHotEncoder(sparse=False, output_type='cudf')

encoded = le.fit_transform(parking_locations['ParkingArea_concat']).to_frame()
encoded = ohe.fit_transform(encoded)
encoded_col_names = [f'parkingArea_cat_{i}' for i in range(encoded.shape[1])]
encoded = cudf.DataFrame(encoded, columns=encoded_col_names)
for col in encoded_col_names:
    encoded[col] = encoded[col].astype('float32')

parking_locations = parking_locations.join(encoded)

In [7]:
parking_locations = dask_cudf.from_cudf(parking_locations, npartitions=8)
bc.create_table('parking_locations', parking_locations)

# Featurize parking transactions

In [8]:
parking_transactions = (
    bc.sql('''
        SELECT A.*
            , TIMESTAMPADD(HOUR, -1, A.OccupancyDateTime) AS time_prior_1h
            , TIMESTAMPADD(DAY,  -1, A.OccupancyDateTime) AS time_prior_1d
            , TIMESTAMPADD(DAY,  -7, A.OccupancyDateTime) AS time_prior_7d
        FROM parking_transactions AS A
        WHERE OccupancyDateTime >= '2019-06-01'
    ''')
)

bc.create_table('parking_transactions', parking_transactions)

In [9]:
parking_transactions_agg = bc.sql('''
    SELECT SourceElementKey
        , transaction_year
        , transaction_month
        , transaction_day
        , transaction_hour
        , AVG(CAST(PaidOccupancy AS FLOAT)) AS average_occupancy
    FROM (
        SELECT A.*
            , YEAR(OccupancyDateTime) AS transaction_year 
            , MONTH(OccupancyDateTime) AS transaction_month
            , DAYOFMONTH(OccupancyDateTime) AS transaction_day
            , HOUR(OccupancyDateTime) AS transaction_hour
        FROM parking_transactions AS A
    ) AS outer_query
    GROUP BY SourceElementKey
        , transaction_year
        , transaction_month
        , transaction_day
        , transaction_hour
    ''')

bc.create_table('parking_transactions_agg', parking_transactions_agg)

In [10]:
dataset_for_training = bc.sql('''
    WITH temp_query_prior_1h AS (
        SELECT A.SourceElementKey
            , A.PaidOccupancy AS Label
            , A.OccupancyDateTime
            , B.average_occupancy as AvgOccupancy_prior_1h
        FROM (
            SELECT SourceElementKey
                , PaidOccupancy
                , OccupancyDateTime
                , YEAR(time_prior_1h) AS transaction_year 
                , MONTH(time_prior_1h) AS transaction_month
                , DAYOFMONTH(time_prior_1h) AS transaction_day
                , HOUR(time_prior_1h) AS transaction_hour
            FROM parking_transactions) AS A
        LEFT OUTER JOIN parking_transactions_agg AS B
            ON A.SourceElementKey = B.SourceElementKey
                AND A.transaction_year = B.transaction_year
                AND A.transaction_month = B.transaction_month
                AND A.transaction_day = B.transaction_day
                AND A.transaction_hour = B.transaction_hour
    )
    , temp_query_prior_1d AS (
        SELECT A.SourceElementKey
            , A.PaidOccupancy AS Label
            , A.OccupancyDateTime
            , B.average_occupancy as AvgOccupancy_prior_1d
        FROM (
            SELECT SourceElementKey
                , PaidOccupancy
                , OccupancyDateTime
                , YEAR(time_prior_1d) AS transaction_year 
                , MONTH(time_prior_1d) AS transaction_month
                , DAYOFMONTH(time_prior_1d) AS transaction_day
                , HOUR(time_prior_1d) AS transaction_hour
            FROM parking_transactions) AS A
        LEFT OUTER JOIN parking_transactions_agg AS B
            ON A.SourceElementKey = B.SourceElementKey
                AND A.transaction_year = B.transaction_year
                AND A.transaction_month = B.transaction_month
                AND A.transaction_day = B.transaction_day
                AND A.transaction_hour = B.transaction_hour
    )
    , temp_query_prior_7d AS (
        SELECT A.SourceElementKey
            , A.PaidOccupancy AS Label
            , A.OccupancyDateTime
            , B.average_occupancy as AvgOccupancy_prior_7d
        FROM (
            SELECT SourceElementKey
                , PaidOccupancy
                , OccupancyDateTime
                , YEAR(time_prior_7d) AS transaction_year 
                , MONTH(time_prior_7d) AS transaction_month
                , DAYOFMONTH(time_prior_7d) AS transaction_day
                , HOUR(time_prior_7d) AS transaction_hour
            FROM parking_transactions) AS A
        LEFT OUTER JOIN parking_transactions_agg AS B
            ON A.SourceElementKey = B.SourceElementKey
                AND A.transaction_year = B.transaction_year
                AND A.transaction_month = B.transaction_month
                AND A.transaction_day = B.transaction_day
                AND A.transaction_hour = B.transaction_hour
    )
    SELECT hr_1.* 
        , d_1.AvgOccupancy_prior_1d
        , d_7.AvgOccupancy_prior_7d
    FROM temp_query_prior_1h AS hr_1
    LEFT OUTER JOIN temp_query_prior_1d AS d_1
        ON hr_1.SourceElementKey = d_1.SourceElementKey
            AND hr_1.Label = d_1.Label
            AND hr_1.OccupancyDateTime = d_1.OccupancyDateTime
    LEFT OUTER JOIN temp_query_prior_7d AS d_7
        ON hr_1.SourceElementKey = d_7.SourceElementKey
            AND hr_1.Label = d_7.Label
            AND hr_1.OccupancyDateTime = d_7.OccupancyDateTime
    
''').repartition(npartitions=8)

In [11]:
dataset_for_training.head()

,SourceElementKey,Label,OccupancyDateTime,AvgOccupancy_prior_1h,AvgOccupancy_prior_1d,AvgOccupancy_prior_7d
0,75865,1,2019-06-28 10:07:00,1.750000,3.266667,2.500000
1,51489,4,2019-06-28 17:32:00,4.233333,4.433333,5.666667
2,68910,4,2019-06-28 13:40:00,4.233333,4.400000,4.066667
3,58513,5,2019-06-28 15:06:00,3.283333,4.933333,3.233333
4,63125,10,2019-06-28 09:22:00,5.316667,4.533333,3.666667


In [12]:
dataset_for_training = (
    dataset_for_training[[
        'SourceElementKey'
        , 'Label'
        , 'AvgOccupancy_prior_1h'
        , 'AvgOccupancy_prior_1d'
        , 'AvgOccupancy_prior_7d'
    ]]
    .merge(parking_locations[['SourceElementKey'] + encoded_col_names], on=['SourceElementKey'])
    .dropna()
    .drop(columns=['SourceElementKey'])
)
dataset_for_training.head()

,Label,AvgOccupancy_prior_1h,AvgOccupancy_prior_1d,AvgOccupancy_prior_7d,parkingArea_cat_0,parkingArea_cat_1,parkingArea_cat_2,parkingArea_cat_3,parkingArea_cat_4,parkingArea_cat_5,...,parkingArea_cat_23,parkingArea_cat_24,parkingArea_cat_25,parkingArea_cat_26,parkingArea_cat_27,parkingArea_cat_28,parkingArea_cat_29,parkingArea_cat_30,parkingArea_cat_31,parkingArea_cat_32
0,3,4.133333,5.866667,7.383333,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,1.700000,2.016667,2.633333,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,2.833333,0.400000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,3.700000,4.050000,4.633333,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,5.066667,4.350000,4.350000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def random_column(df):
    frame_len = len(df)
    df['random'] = cp.random.rand(frame_len)
    return df
    
dataset_for_training['Label'] = dataset_for_training['Label'].astype('float32')
dataset_for_training = dataset_for_training.map_partitions(random_column).compute()

train_X = dataset_for_training.query('random < 0.7')[[
    'AvgOccupancy_prior_1h'
    , 'AvgOccupancy_prior_1d'
    , 'AvgOccupancy_prior_7d'
] + encoded_col_names]

train_y = dataset_for_training.query('random < 0.7')['Label']

test_X = dataset_for_training.query('random >= 0.7')[[
    'AvgOccupancy_prior_1h'
    , 'AvgOccupancy_prior_1d'
    , 'AvgOccupancy_prior_7d'
] + encoded_col_names]

test_y = dataset_for_training.query('random >= 0.7')['Label']

In [14]:
print(f'Full size: {len(dataset_for_training):,}, size of training: {len(train_y):,}, size of testing: {len(test_y):,}')

Full size: 13,802,884, size of training: 9,666,248, size of testing: 4,136,636


In [15]:
train_X.head()

,AvgOccupancy_prior_1h,AvgOccupancy_prior_1d,AvgOccupancy_prior_7d,parkingArea_cat_0,parkingArea_cat_1,parkingArea_cat_2,parkingArea_cat_3,parkingArea_cat_4,parkingArea_cat_5,parkingArea_cat_6,...,parkingArea_cat_23,parkingArea_cat_24,parkingArea_cat_25,parkingArea_cat_26,parkingArea_cat_27,parkingArea_cat_28,parkingArea_cat_29,parkingArea_cat_30,parkingArea_cat_31,parkingArea_cat_32
2,7.950000,12.000000,8.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.700000,1.733333,2.233333,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3.733333,0.466667,1.716667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2.700000,0.583333,1.533333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,5.116667,3.266667,2.583333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from cuml import RandomForestRegressor
from cuml.metrics.regression import r2_score

In [17]:
rfr = RandomForestRegressor(
    n_estimators=10
    , max_depth=20
    , min_rows_per_node=50
    , verbose=2
)

In [18]:
rfr.fit(train_X, train_y)

RandomForestRegressor(split_criterion=2, accuracy_metric='r2', handle=<cuml.raft.common.handle.Handle object at 0x7f2948a2d550>, verbose=2, output_type='cudf')

In [19]:
rfr.predict(test_X)

0          0.849858
1          2.866051
2          3.497519
3          1.224828
4          2.513590
             ...   
4136631    1.106243
4136632    2.784359
4136633    5.989009
4136634    0.205775
4136635    0.205775
Length: 4136636, dtype: float32

In [20]:
r2_score(test_y, rfr.predict(test_X))

0.7532678842544556

# Featurize parking locations

In [21]:
def extractLon(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[0]
    return lon

def extractLat(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[1]
    return lon
    
parking_locations['longitude'] = extractLon(parking_locations['Location']).astype('float32')
parking_locations['latitude'] = extractLat(parking_locations['Location']).astype('float32')

parking_locations[['Location', 'longitude', 'latitude']].head()

,Location,longitude,latitude
0,POINT (-122.35213217 47.61658464),-122.352135,47.616585
1,POINT (-122.35550455 47.62268904),-122.355507,47.622688
2,POINT (-122.33428859 47.60047073),-122.334290,47.600471
3,POINT (-122.31945954 47.61196395),-122.319458,47.611965
4,POINT (-122.31642786 47.66400789),-122.316429,47.664009


## As crow flies vs as people walk

![fff](https://miro.medium.com/max/1400/1*zmMox5tjq93AiBZdPRG3gg.png)

### Read in the graph data
Thanks to John Murray for analyzing the map of King County roads and producing the data we will now use.

#### Download and unpack the data

In [22]:
import os

directory  = os.path.exists('../../data')
archive    = os.path.exists('../../data/king_county_road_graph_20190909.tar.gz')
file_graph = os.path.exists('../../data/king_county_road_graph_20190909.csv')
file_nodes = os.path.exists('../../data/king_county_road_nodes_20190909.csv')

if not directory:
    os.mkdir('../data')

if not archive:
    import wget, shutil
    
    wget.download('http://tomdrabas.com/data/seattle_parking/king_county_road_graph_20190909.tar.gz')
    shutil.move('king_county_road_graph_20190909.tar.gz', '../../data/king_county_road_graph_20190909.tar.gz')
    
if not file_graph or not file_nodes:
    import tarfile

    tf = tarfile.open('../../data/king_county_road_graph_20190909.tar.gz')
    tf.extractall(path='../../data/')

#### Let's read the King County road data

In [23]:
road_graph_data = cudf.read_csv('../../data/seattle_parking/king_county_road_graph_20190909.csv')
road_graph_data['node1'] = road_graph_data['node1'].astype('int32')
road_graph_data['node2'] = road_graph_data['node2'].astype('int32')
road_graph_data['LENGTH'] = road_graph_data['LENGTH'] * 3 # convert to feet as the LENGHT was given in yards

In [24]:
road_nodes = cudf.read_csv('../../data/seattle_parking/king_county_road_nodes_20190909.csv')
road_nodes['NodeID'] = road_nodes['NodeID'].astype('int32')

Store the maximum of the `NodeID` so we can later append the additional nodes that will be perpendicular to the actual parking locations. We also specify the offset - this will be used to append parking nodes.

In [25]:
offset = 100000
nodeId = road_nodes['NodeID'].max()                       ## so we can number the parking nodes properly (since we'll be adding a perpendicular projections)
parking_nodes_idx = road_nodes['NodeID'].max() + offset   ## retain it so we can later filter the results to only parking locations
nodeId

127380

Move all the parking locations to host (via `.to_pandas()` method) so we can loop through all the ~1500 parking locations. Here, we also create an empty DataFrame that will hold the parking location nodes.

In [26]:
locations = parking_locations.compute().to_pandas().to_dict('records')
parking_locations_nodes = cudf.DataFrame(columns=['NodeID', 'Lon', 'Lat', 'SourceElementKey'])
added_location_edges    = cudf.DataFrame(columns=['node1', 'node2', 'LENGTH'])

Let's process the parking data. The kernel below finds equations of two lines:

1. Line that goes through road intersections
2. Line that is perpendicular to (1) and goes through the parking location.

![lll](https://miro.medium.com/max/1296/1*4Sg3alMbrT3DndIzM9dS4Q.gif)

Ultimately, we are finind the intersection of these two lines -- we call it the `PROJ` point below.

In [27]:
def kernel_find_projection(Lon_x, Lat_x, Lon_y, Lat_y, Lon_PROJ, Lat_PROJ, Lon_REF, Lat_REF):
    for i, (lon_x, lat_x, lon_y, lat_y) in enumerate(zip(Lon_x, Lat_x, Lon_y, Lat_y)):
        # special case where A and B have the same LON i.e. vertical line
        if lon_x == lon_y:
            Lon_PROJ[i] = lon_x
            Lat_PROJ[i] = Lat_REF    
        else:
            # find slope
            a_xy = (lat_x - lat_y) / float(lon_x - lon_y)

            # special case where A and B have the same LAT i.e. horizontal line
            if a_xy == 0:
                Lon_PROJ[i] = Lon_REF
                Lat_PROJ[i] = lat_x
            else: 
                # if neither of the above special cases apply
                # find the equation of the perpendicular line
                a_R  = -1 / a_xy                    ### SLOPE

                # find intersections
                b_xy = lat_x - a_xy * lon_x
                b_R  = Lat_REF - a_R  * Lon_REF

                # find the coordinates
                Lon_PROJ[i] = (b_R - b_xy) / (a_xy - a_R)
                Lat_PROJ[i] = a_R * Lon_PROJ[i] + b_R

In [28]:
%%time
parking_locations_cnt = len(locations)
print('Number of parking locations: {0:,}'.format(parking_locations_cnt))

for i, loc in enumerate(locations):
    if i % 100 == 0:
        print('Processed: {0:,} ({1:.2%}) nodes'.format(i, i/float(parking_locations_cnt)))
        
    #### INCREASE THE COUNTER AND GET THE REFERENCE POINT
    nodeId = nodeId + 1
    lat_r = loc['latitude']
    lon_r = loc['longitude']

    #### APPEND GEO COORDINATES TO INTERSECTION AND SUBSET DOWN THE DATASET
    #### TO POINTS WITHIN ~2000ft FROM PARKING SPOT
    paths = (
        road_graph_data
        .rename(columns={'node1': 'NodeID'})
        .merge(road_nodes[['NodeID', 'Lat', 'Lon']], on='NodeID', how='left')
        .rename(columns={'NodeID': 'node1', 'node2': 'NodeID'})
        .merge(road_nodes[['NodeID', 'Lat', 'Lon']], on='NodeID', how='left')
        .rename(columns={'NodeID': 'node2'})
        .query('Lat_x >= (@lat_r - 0.005) and Lat_x <= (@lat_r + 0.005)')
        .query('Lon_x >= (@lon_r - 0.005) and Lon_x <= (@lon_r + 0.005)')
        .query('Lat_y >= (@lat_r - 0.005) and Lat_y <= (@lat_r + 0.005)')
        .query('Lon_y >= (@lon_r - 0.005) and Lon_y <= (@lon_r + 0.005)')
    )

    #### APPEND THE PARKING LOCATION SO WE CAN CALCULATE DISTANCES
    paths['Lon_REF'] = loc['longitude']
    paths['Lat_REF'] = loc['latitude']

    paths = paths.apply_rows(
        kernel_find_projection
        , incols  = ['Lon_x', 'Lat_x', 'Lon_y', 'Lat_y', 'Lon_REF', 'Lat_REF']
        , outcols = {'Lon_PROJ': np.float64, 'Lat_PROJ': np.float64}
        , kwargs  = {'Lon_REF': loc['longitude'], 'Lat_REF': loc['latitude']}
    )

    #### CALCULATE THE DISTANCES SO WE CAN CHECK IF THE PROJ POINT IS BETWEEN ROAD NODES
    paths['Length_x_PROJ'] = cuspatial.haversine_distance(
              paths['Lon_x']
            , paths['Lat_x']
            , paths['Lon_PROJ']
            , paths['Lat_PROJ'])# * 0.621371 * 5280
    paths['Length_x_PROJ'] = paths['Length_x_PROJ'] * 0.621371 * 5280

    paths['Length_y_PROJ'] = cuspatial.haversine_distance(
              paths['Lon_y']
            , paths['Lat_y']
            , paths['Lon_PROJ']
            , paths['Lat_PROJ']) 
    paths['Length_y_PROJ'] = paths['Length_y_PROJ'] * 0.621371 * 5280

    paths['Length_REF_PROJ'] = cuspatial.haversine_distance(
              paths['Lon_REF']
            , paths['Lat_REF']
            , paths['Lon_PROJ']
            , paths['Lat_PROJ']) 
    paths['Length_REF_PROJ'] = paths['Length_REF_PROJ'] * 0.621371 * 5280

    #### SELECT THE POINTS THAT A LESS THAN OR EQAL TO TOTAL LENGTH OF THE EDGE (WITHIN 1 ft)
    paths['PROJ_between'] = (paths['Length_x_PROJ'] + paths['Length_y_PROJ']) <= (paths['LENGTH'] + 1)
    
    #### SELECT THE CLOSEST
    closest = (
        paths
        .query('PROJ_between')
        .nsmallest(1, 'Length_REF_PROJ')
        .to_pandas()
        .to_dict('records')[0]
    )

    # add nodes
    nodes =    cudf.DataFrame({
          'NodeID': [nodeId + offset, nodeId]
        , 'Lon':    [closest['Lon_REF'], closest['Lon_PROJ']]
        , 'Lat':    [closest['Lat_REF'], closest['Lat_PROJ']]
        , 'SourceElementKey': [loc['SourceElementKey'], None]
    })

    parking_locations_nodes = cudf.concat([parking_locations_nodes, nodes])

    # add edges (bi-directional)
    edges = cudf.DataFrame({
          'node1':  [nodeId, nodeId, nodeId, closest['node1'], closest['node2'], nodeId + offset]
        , 'node2':  [closest['node1'], closest['node2'], nodeId + offset, nodeId, nodeId, nodeId]
        , 'LENGTH': [
              closest['Length_x_PROJ'], closest['Length_y_PROJ'], closest['Length_REF_PROJ']
            , closest['Length_x_PROJ'], closest['Length_y_PROJ'], closest['Length_REF_PROJ']
        ]
    })

    added_location_edges = cudf.concat([added_location_edges, edges]) ## append to the temp DataFrame

print('Finished processing...')

Number of parking locations: 1,528
Processed: 0 (0.00%) nodes
Processed: 100 (6.54%) nodes
Processed: 200 (13.09%) nodes
Processed: 300 (19.63%) nodes
Processed: 400 (26.18%) nodes
Processed: 500 (32.72%) nodes
Processed: 600 (39.27%) nodes
Processed: 700 (45.81%) nodes
Processed: 800 (52.36%) nodes
Processed: 900 (58.90%) nodes
Processed: 1,000 (65.45%) nodes
Processed: 1,100 (71.99%) nodes
Processed: 1,200 (78.53%) nodes
Processed: 1,300 (85.08%) nodes
Processed: 1,400 (91.62%) nodes
Processed: 1,500 (98.17%) nodes
Finished processing...
CPU times: user 1min 22s, sys: 11.3 s, total: 1min 33s
Wall time: 1min 31s


In [29]:
road_graph_data.head()

,node1,node2,LENGTH
0,89108,27652,15.08475
1,27652,89108,15.08475
2,27652,122930,337.25100
3,122930,27652,337.25100
4,36778,36779,144.74250


In [30]:
road_nodes = (
    cudf
    .concat([road_nodes[['NodeID', 'Lon', 'Lat']], parking_locations_nodes])
    .reset_index(drop=True)
)

Now we can find the nearest intersections from the Space Needle!

In [31]:
location = {'latitude': 47.620422, 'longitude': -122.349358}
date = dttm.datetime.strptime('2019-06-21 13:21:00', '%Y-%m-%d %H:%M:%S')

In [32]:
bc.create_table('parking_transactions'
    , bc.sql(f'''
        SELECT *
            , YEAR(OccupancyDateTime) AS transaction_year 
            , MONTH(OccupancyDateTime) AS transaction_month
            , DAYOFMONTH(OccupancyDateTime) AS transaction_day
            , HOUR(OccupancyDateTime) AS transaction_hour
        FROM parking_transactions
    ''')
)

In [33]:
road_nodes['Lon_REF'] = location['longitude']
road_nodes['Lat_REF'] = location['latitude']

road_nodes['Distance'] = cuspatial.haversine_distance(
          road_nodes['Lon']
        , road_nodes['Lat']
        , road_nodes['Lon_REF']
        , road_nodes['Lat_REF']) 
road_nodes['Distance'] = road_nodes['Distance'] * 0.621371 * 5280

space_needle_to_nearest_intersection = road_nodes.nsmallest(5, 'Distance') ### Space Needle is surrounded by around 5 road intersections hence we add 5
space_needle_to_nearest_intersection_dist = space_needle_to_nearest_intersection['Distance'].to_array()[0]

space_needle_to_nearest_intersection['node1'] = nodeId + 2
space_needle_to_nearest_intersection = (
    space_needle_to_nearest_intersection
    .rename(columns={'NodeID': 'node2', 'Distance': 'LENGTH'})
    [['node1', 'node2', 'LENGTH']]
)

road_graph_data = cudf.concat([space_needle_to_nearest_intersection, added_location_edges, road_graph_data])
space_needle_to_nearest_intersection ### SHOW THE EDGES

,node1,node2,LENGTH
47756,128910,47757,210.932442
80448,128910,80449,233.601728
108797,128910,108798,248.140475
96739,128910,96740,270.755304
47827,128910,47828,276.651357


### The road graph

In [34]:
road_graph_data = road_graph_data.reset_index(drop=True)
road_graph_data['node1'] = road_graph_data['node1'].astype('int32')
road_graph_data['node2'] = road_graph_data['node2'].astype('int32')

In [35]:
g = cugraph.Graph()
g.from_cudf_edgelist(
    road_graph_data
    , source='node1'
    , destination='node2'
    , edge_attr='LENGTH'
    , renumber=False
)

Now we can use the `.sssp(...)` method from `cugraph` to find the shortest distances to parking spots from the Space Needle!

In [36]:
all_distances = cugraph.sssp(g, nodeId + 2)
distances = all_distances.query('vertex > @parking_nodes_idx and distance < 1000')
distances

,vertex,distance,predecessor
227436,227436,446.496776,127436
227584,227584,561.281059,127584
227667,227667,773.327847,127667
227819,227819,469.123120,127819
227872,227872,930.104864,127872
227902,227902,772.298512,127902
227903,227903,767.847642,127903
227982,227982,957.997564,127982
228347,228347,517.336044,128347
228630,228630,953.941558,128630


#### INFERENCE

In [37]:
inference = distances.merge(parking_locations_nodes, left_on='vertex', right_on='NodeID')
inference['OccupancyDateTime'] = date
inference['time_prior_1h'] = date + dttm.timedelta(hours=-1)
inference['time_prior_1d'] = date + dttm.timedelta(days=-1)
inference['time_prior_7d'] = date + dttm.timedelta(days=-7)
bc.create_table('inference', inference)
inference

,vertex,distance,predecessor,NodeID,Lon,Lat,SourceElementKey,OccupancyDateTime,time_prior_1h,time_prior_1d,time_prior_7d
0,228718,994.409792,128718,228718,-122.349480,47.618061,54985,2019-06-21 13:21:00,2019-06-21 12:21:00,2019-06-20 13:21:00,2019-06-14 13:21:00
1,228347,517.336044,128347,228347,-122.348793,47.619144,51494,2019-06-21 13:21:00,2019-06-21 12:21:00,2019-06-20 13:21:00,2019-06-14 13:21:00
2,228630,953.941558,128630,228630,-122.348572,47.618008,28961,2019-06-21 13:21:00,2019-06-21 12:21:00,2019-06-20 13:21:00,2019-06-14 13:21:00
3,227584,561.281059,127584,227584,-122.349037,47.619064,51493,2019-06-21 13:21:00,2019-06-21 12:21:00,2019-06-20 13:21:00,2019-06-14 13:21:00
4,227819,469.123120,127819,227819,-122.348175,47.619804,11133,2019-06-21 13:21:00,2019-06-21 12:21:00,2019-06-20 13:21:00,2019-06-14 13:21:00
5,227436,446.496776,127436,227436,-122.348259,47.619678,11134,2019-06-21 13:21:00,2019-06-21 12:21:00,2019-06-20 13:21:00,2019-06-14 13:21:00
6,227982,957.997564,127982,227982,-122.349297,47.617996,54986,2019-06-21 13:21:00,2019-06-21 12:21:00,2019-06-20 13:21:00,2019-06-14 13:21:00
7,227872,930.104864,127872,227872,-122.348465,47.618145,28962,2019-06-21 13:21:00,2019-06-21 12:21:00,2019-06-20 13:21:00,2019-06-14 13:21:00
8,227902,772.298512,127902,227902,-122.346954,47.619808,33741,2019-06-21 13:21:00,2019-06-21 12:21:00,2019-06-20 13:21:00,2019-06-14 13:21:00
9,227903,767.847642,127903,227903,-122.346962,47.619667,33742,2019-06-21 13:21:00,2019-06-21 12:21:00,2019-06-20 13:21:00,2019-06-14 13:21:00


In [38]:
inference = bc.sql(f'''
    WITH temp_query_prior_1h AS (
        SELECT A.SourceElementKey
            , A.OccupancyDateTime
            , B.average_occupancy as AvgOccupancy_prior_1h
        FROM (
            SELECT SourceElementKey
                , OccupancyDateTime
                , YEAR(time_prior_1h) AS transaction_year 
                , MONTH(time_prior_1h) AS transaction_month
                , DAYOFMONTH(time_prior_1h) AS transaction_day
                , HOUR(time_prior_1h) AS transaction_hour
            FROM inference) AS A
        LEFT OUTER JOIN parking_transactions_agg AS B
            ON A.SourceElementKey = B.SourceElementKey
                AND A.transaction_year = B.transaction_year
                AND A.transaction_month = B.transaction_month
                AND A.transaction_day = B.transaction_day
                AND A.transaction_hour = B.transaction_hour
    )
    , temp_query_prior_1d AS (
        SELECT A.SourceElementKey
            , A.OccupancyDateTime
            , B.average_occupancy as AvgOccupancy_prior_1d
        FROM (
            SELECT SourceElementKey
                , OccupancyDateTime
                , YEAR(time_prior_1d) AS transaction_year 
                , MONTH(time_prior_1d) AS transaction_month
                , DAYOFMONTH(time_prior_1d) AS transaction_day
                , HOUR(time_prior_1d) AS transaction_hour
            FROM inference) AS A
        LEFT OUTER JOIN parking_transactions_agg AS B
            ON A.SourceElementKey = B.SourceElementKey
                AND A.transaction_year = B.transaction_year
                AND A.transaction_month = B.transaction_month
                AND A.transaction_day = B.transaction_day
                AND A.transaction_hour = B.transaction_hour
    )
    , temp_query_prior_7d AS (
        SELECT A.SourceElementKey
            , A.OccupancyDateTime
            , B.average_occupancy as AvgOccupancy_prior_7d
        FROM (
            SELECT SourceElementKey
                , OccupancyDateTime
                , YEAR(time_prior_7d) AS transaction_year 
                , MONTH(time_prior_7d) AS transaction_month
                , DAYOFMONTH(time_prior_7d) AS transaction_day
                , HOUR(time_prior_7d) AS transaction_hour
            FROM inference) AS A
        LEFT OUTER JOIN parking_transactions_agg AS B
            ON A.SourceElementKey = B.SourceElementKey
                AND A.transaction_year = B.transaction_year
                AND A.transaction_month = B.transaction_month
                AND A.transaction_day = B.transaction_day
                AND A.transaction_hour = B.transaction_hour
    )
    SELECT hr_1.* 
        , d_1.AvgOccupancy_prior_1d
        , d_7.AvgOccupancy_prior_7d
    FROM temp_query_prior_1h AS hr_1
    LEFT OUTER JOIN temp_query_prior_1d AS d_1
        ON hr_1.SourceElementKey = d_1.SourceElementKey
            AND hr_1.OccupancyDateTime = d_1.OccupancyDateTime
    LEFT OUTER JOIN temp_query_prior_7d AS d_7
        ON hr_1.SourceElementKey = d_7.SourceElementKey
            AND hr_1.OccupancyDateTime = d_7.OccupancyDateTime
    ORDER BY SourceElementKey
    
''').compute()

In [39]:
inference_X = (
    inference[[
        'SourceElementKey'
        , 'AvgOccupancy_prior_1h'
        , 'AvgOccupancy_prior_1d'
        , 'AvgOccupancy_prior_7d'
    ]]
    .merge(parking_locations[['SourceElementKey'] + encoded_col_names].compute(), on=['SourceElementKey'])
    .sort_values(by='SourceElementKey')
    .dropna()
    .drop(columns=['SourceElementKey'])
)
inference_X

,AvgOccupancy_prior_1h,AvgOccupancy_prior_1d,AvgOccupancy_prior_7d,parkingArea_cat_0,parkingArea_cat_1,parkingArea_cat_2,parkingArea_cat_3,parkingArea_cat_4,parkingArea_cat_5,parkingArea_cat_6,...,parkingArea_cat_23,parkingArea_cat_24,parkingArea_cat_25,parkingArea_cat_26,parkingArea_cat_27,parkingArea_cat_28,parkingArea_cat_29,parkingArea_cat_30,parkingArea_cat_31,parkingArea_cat_32
10,6.200000,9.016666,8.616667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
0,1.450000,2.066667,3.583333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,6.533333,4.633333,5.766667,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.000000,2.233333,2.750000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,5.316667,4.183333,5.566667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,7.816667,9.900000,7.233333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4.000000,6.350000,6.300000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,16.400000,11.966666,16.350000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,5.100000,4.633333,5.966667,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.816667,6.133333,6.466667,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
inference.join(rfr.predict(inference_X).to_frame('prediction'))

,SourceElementKey,OccupancyDateTime,AvgOccupancy_prior_1h,AvgOccupancy_prior_1d,AvgOccupancy_prior_7d,prediction
0,11133,2019-06-21 13:21:00,6.200000,9.016666,8.616667,6.803786
1,11134,2019-06-21 13:21:00,1.450000,2.066667,3.583333,2.205611
2,28961,2019-06-21 13:21:00,6.533333,4.633333,5.766667,5.577992
3,28962,2019-06-21 13:21:00,2.000000,2.233333,2.750000,2.526133
4,33741,2019-06-21 13:21:00,5.316667,4.183333,5.566667,5.694440
5,33742,2019-06-21 13:21:00,7.816667,9.900000,7.233333,16.109140
6,51493,2019-06-21 13:21:00,4.000000,6.350000,6.300000,4.873112
7,51494,2019-06-21 13:21:00,16.400000,11.966666,16.350000,16.109140
8,54985,2019-06-21 13:21:00,5.100000,4.633333,5.966667,5.577992
9,54986,2019-06-21 13:21:00,5.816667,6.133333,6.466667,5.929451


`cugraph` returns a DataFrame with vertex, distance to that vertex, and the total distance traveled to that vertex from the `nodeId + 1` node -- the Space Needle. Here, we unfold the full path.

In [41]:
# unfold -- create the whole path
closest_node = nodeId + 2
parking_cnt = distances['vertex'].count()

for i in range(parking_cnt):
    print('Processing record: {0}'.format(i))
    parking_node = distances.iloc[i]

    vertex = int(parking_node['vertex'])
    predecessor = int(parking_node['predecessor'])
    
    if i == 0:
        paths = all_distances.query('vertex == @vertex')
    else:
        paths = cudf.concat([all_distances.query('vertex == @vertex'), paths])

    while vertex != closest_node:
        temp = all_distances.query('vertex == @predecessor')
        paths = cudf.concat([temp, paths])
        predecessor = temp['predecessor'].to_array()[0]
        vertex = temp['vertex'].to_array()[0]

Processing record: 0
Processing record: 1
Processing record: 2
Processing record: 3
Processing record: 4
Processing record: 5
Processing record: 6
Processing record: 7
Processing record: 8
Processing record: 9
Processing record: 10


In [42]:
paths

,vertex,distance,predecessor
128910,128910,0.000000,-1
47828,47828,276.651357,128910
127584,127584,526.834962,47828
47830,47830,703.043055,127584
47832,47832,770.523855,47830
...,...,...,...
227584,227584,561.281059,127584
128910,128910,0.000000,-1
47828,47828,276.651357,128910
127436,127436,424.530732,47828
